# Programming with Big Data

In this workbook, we will gain a greater understanding of the map-reduce model of computation by implementing our own system for running map reduce jobs.

## Mapping functions in general

In computer science, the "map" function is a higher-order function, which means that it is a function that takes another function as one of its arguments. In the case of map, arguments are a function and a collection, and the result is a new collection which results

In [ ]:
result1 = map(len, ["hi", "hello", "hola", "ni hao"])

def square(x):
    return x * x

result2 = map(square, [0, 1, 2, 3, 4])

result3 = map(lambda x: (x.upper(), x.lower()), ["Bye", "Adios", "Zai Jian"])

In each of these assignments, we are mapping a function (the first argument) over a collection (the second argument). Here we have used lists as our collections. The function can be built-in, it can be a named function that we define, or can be an inline function defined using python's `lambda` keyword.

Let's take a look at the result.

In [ ]:
print(result1)

What happened? For efficiency's sake, python doesn't actually compute the final result until it is needed elsewhere in the program. One way to force evaluation is to construct a new collection using the result of the map.

In [ ]:
# a list containing the lengths of the words hi, hello, hola, and ni hao.
list(result1)

Let's print the other results.

In [ ]:
# a list containing the squares of the numbers 0 through 4
print(list(result2))

# a list of pairs
print(list(result3))

An important aspect of the `map` function is that each application of the function argment to an element in the collection argument is independent of all other applications. That means that all these function applications can happen in parallel.

## Reducing collections in general

To reduce a collection you need a function of two arguments:
* The first argument will be the output from a previous application of the function (or an initial value)
* The second argument will be an element of the collection

It will be easier to understand if we give an example. Given a list of numbers `[1, 5, 2, 6, 2]` we can represent the sum as an application of `reduce`. In this setup:
* The collection is the list `[1, 5, 2, 6, 2]`
* The function is simple pairwise addition
* The initial value is 0

Let's see an example in code.

In [ ]:
from functools import reduce

coll = [1, 5, 2, 6, 2]

def add2(x, y):
    return x + y

initial_value = 0

reduce(add2, coll, initial_value)

As a second example, we can count the occurences of characters in a sstring using the `reduce` function. In this case:

* The collection is the string (python treats strings as collection of their constituent characters)
* The function will take a python dictionary and a character as arguments. The dictionary will use characters as keys and counts as values. The function will look up a character in the dictionary, increment its count, and return the result.
* The initial value is an empty dictionary object

In [ ]:
coll = "The rain in Spain falls mainly on the plain."

def count_character(counts, char):
    """If char already appears in counts then increment the associated
    value. Otherwise, set the associated value to 1."""
    
    if char in counts:
        counts[char] += 1
    else:
        counts[char] = 1
        
    return counts

# we don't need to define a variable named initial_value,
# we can just pass an empty dictionary to the reduce function

reduce(count_character, coll, {})

As a final example, we can compute the maximum of a collection of numbers. In this case, the binary function is just the built-in `max` function. What about the initial value? One option would be to use python's representation of negative infinity, because it will be smaller than any other number that appears in our collection.

There is a second option that takes advantage of the domain and range of the `max` function. Note that there is a difference between `max` and `add2` on the one hand and `count_character` on the other hand. In the case of `max` and `add2`, the output of those functions belongs to the same domain as the collections they are applied to. In the case of `count_character`, however, the output is a dictionary but the collection it is applied to contains characters.

When your reducing function produces values in the same domain as the element of the collection it is applied to, you can actually leave out the initial value. python will then take an initial value from the collection itself.

In [ ]:
# compute the maximum of the collection

reduce(max, [79, 81, 54, 101, 25, 95])

We can rewrite the summation example without using an initial value.

In [ ]:
reduce(add2, [1, 5, 2, 6, 2])

As with the `map` function, the `reduce` function ecapsulates a general paradigm of computation in the form of a higher-order function. In terms of the map-reduce paradigm, reducing will be used to produce a final answer after the mapping steps have been applied in parallel.

# Putting it together



In [ ]:
def map_reduce(mapfn, reducefn, data, n):
    
    # step 1: divide data into n chunks
    
    size = len(data)
    chunk_size = int(size / n + 1) # the "+ 1" rounds up
    chunks = [words[k:(k + chunk_size)] for k in range(0, size, chunk_size)]
    
    # step 2: apply mapfn to each chunk

    map_result = [map(mapfn, chunk) for chunk in chunks]
    
    # step 3: reduce the chunks into a single output value

    accumulator = {}
    for chunk in map_result:
        reduce(reducefn, chunk, accumulator)
    
    return accumulator

In [ ]:
def map_count(word):
    return (word, 1)

def reduce_count(accumulator, map_result):
    word, n = map_result
    word = word.lower()
    
    if word in accumulator:
        accumulator[word] += n
    else:
        accumulator[word] = n
            
    return accumulator

In [ ]:
address = """
    Four score and seven years ago our fathers brought forth on this continent a new nation, conceived in liberty, and dedicated to the proposition that all men are created equal.

    Now we are engaged in a great civil war, testing whether that nation, or any nation so conceived and so dedicated, can long endure. We are met on a great battlefield of that war. We have come to dedicate a portion of that field, as a final resting place for those who here gave their lives that that nation might live. It is altogether fitting and proper that we should do this.

    But, in a larger sense, we can not dedicate, we can not consecrate, we can not hallow this ground. The brave men, living and dead, who struggled here, have consecrated it, far above our poor power to add or detract. The world will little note, nor long remember what we say here, but it can never forget what they did here. It is for us the living, rather, to be dedicated here to the unfinished work which they who fought here have thus far so nobly advanced. It is rather for us to be here dedicated to the great task remaining before us—that from these honored dead we take increased devotion to that cause for which they gave the last full measure of devotion—that we here highly resolve that these dead shall not have died in vain—that this nation, under God, shall have a new birth of freedom—and that government of the people, by the people, for the people, shall not perish from the earth."""

import re
address_nopunct = re.sub("[^\w\s]", "", address)
words = address_nopunct.split()

map_reduce(map_count, reduce_count, words, 4)